In [33]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import re
import gc
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [34]:
gc.collect()

101

In [35]:
fal_sin=pd.read_csv("/content/fallecidos_sinadef.csv", sep=';' , encoding='latin-1',header=2).iloc[:,1:-4]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
fal_sin.head()

,TIPO SEGURO,SEXO,EDAD,TIEMPO EDAD,ESTADO CIVIL,NIVEL DE INSTRUCCIÓN,COD# UBIGEO DOMICILIO,PAIS DOMICILIO,DEPARTAMENTO DOMICILIO,PROVINCIA DOMICILIO,DISTRITO DOMICILIO,FECHA,AÑO,MES,TIPO LUGAR,INSTITUCION,MUERTE VIOLENTA,NECROPSIA,DEBIDO A (CAUSA A),CAUSA A (CIE-X),DEBIDO A (CAUSA B),CAUSA B (CIE-X),DEBIDO A (CAUSA C),CAUSA C (CIE-X),DEBIDO A (CAUSA D),CAUSA D (CIE-X),DEBIDO A (CAUSA E),CAUSA E (CIE-X),DEBIDO A (CAUSA F),CAUSA F (CIE-X)
0,SIS,MASCULINO,4,AÑOS,SOLTERO,IGNORADO,92-33-21-04-01-000,PERU,SAN MARTIN,MARISCAL CACERES,JUANJUI,2020-01-17,2020,1,EESS,GOBIERNO REGIONAL,SIN REGISTRO,NO SE REALIZÓ NECROPSIA,INSUFICIENCIA RESPIRATORIA AGUDA,J960,NEUMONIA SEVERA,J159,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,PARALISIS CEREBRAL INFANTIL,G809,SIN REGISTRO,SIN REGISTRO
1,SIS,FEMENINO,86,AÑOS,SOLTERO,PRIMARIA INCOMPLETA,92-33-21-03-13-000,PERU,SAN MARTIN,LAMAS,TABALOSOS,2020-01-03,2020,1,DOMICILIO,SIN REGISTRO,SIN REGISTRO,NO SE REALIZÓ NECROPSIA,INFARTO AGUDO DEL MIOCARDIO SIN OTRA ESPECIFIC...,I219,HIPERLIPIDEMIA NO ESPECIFICADA,E785,ATEROSCLEROSIS GENERALIZADA Y LA NO ESPECIFICADA,I709,HIPERTENSION ESENCIAL,I10X,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
2,IGNORADO,MASCULINO,38,AÑOS,IGNORADO,IGNORADO,93-32-00-00-00-000,FILIPINAS,EXTRANJERO,EXTRANJERO,EXTRANJERO,2020-01-19,2020,1,OTRO,SIN REGISTRO,SIN REGISTRO,SI SE REALIZÓ NECROPSIA,EDEMA Y CONGESTION CEREBRAL Y PULMONAR,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
3,USUARIO,FEMENINO,64,AÑOS,SOLTERO,PRIMARIA COMPLETA,92-33-14-01-37-000,PERU,LIMA,LIMA,SAN JUAN DE LURIGANCHO,2020-01-06,2020,1,DOMICILIO,SIN REGISTRO,SIN REGISTRO,SI SE REALIZÓ NECROPSIA,TUBERCULOSIS PULMONAR,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
4,IGNORADO,MASCULINO,20,AÑOS,SOLTERO,IGNORADO,92-33-24-01-02-000,PERU,CALLAO,CALLAO,BELLAVISTA,2020-01-13,2020,1,EESS,GOBIERNO REGIONAL,HOMICIDIO,SI SE REALIZÓ NECROPSIA,SHOCK HIPOVOLEMICO,SIN REGISTRO,TAPONAMIENTO CARDIACO,SIN REGISTRO,TRAUMA TORACICO ABIERTO POR PROYECTIL DE ARMA ...,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO


In [37]:
fal_sin["TIPO SEGURO"].value_counts(dropna=False)

SIS              253923
ESSALUD          156001
IGNORADO          94457
USUARIO            9863
OTROS              9504
PRIVADOS           8699
SANIDAD PNP        8361
SANIDAD NAVAL      1533
SANIDAD EP         1377
SANIDAD FAP        1363
EXONERADO           900
SOAT                 29
SIN REGISTRO          1
Name: TIPO SEGURO, dtype: int64

In [38]:
fal_sin["TIPO SEGURO"].value_counts()

SIS              253923
ESSALUD          156001
IGNORADO          94457
USUARIO            9863
OTROS              9504
PRIVADOS           8699
SANIDAD PNP        8361
SANIDAD NAVAL      1533
SANIDAD EP         1377
SANIDAD FAP        1363
EXONERADO           900
SOAT                 29
SIN REGISTRO          1
Name: TIPO SEGURO, dtype: int64

In [39]:
def  TIPO_SEGURO_FE(x):
    if(x=="SANIDAD PNP"):
      return "SANIDAD"
    if(x=="SANIDAD NAVAL"):
      return "SANIDAD"
    if(x=="SANIDAD EP"):
      return "SANIDAD"
    if(x=="SANIDAD FAP"):
      return "SANIDAD"
    if(x=="SOAT"):
      return "PRIVADOS"
    if(x=="SIS"):
      return "PUBLICO"
    if(x=="ESSALUD"):
      return "PUBLICO"
    return "OTROS"

In [40]:
fal_sin["TIPO SEGURO_FE"]=fal_sin["TIPO SEGURO"].apply(TIPO_SEGURO_FE)

In [41]:
fal_sin["TIPO SEGURO_FE"].value_counts()

PUBLICO     409924
OTROS       123424
SANIDAD      12634
PRIVADOS        29
Name: TIPO SEGURO_FE, dtype: int64

In [42]:
fal_sin["SEXO"].value_counts(dropna=False)

MASCULINO        306169
FEMENINO         239771
INDETERMINADO        67
SIN REGISTRO          4
Name: SEXO, dtype: int64

In [43]:
fal_sin["TIEMPO EDAD"].value_counts(dropna=False)

AÑOS            527537
DIAS              7541
MESES             7047
HORAS             2641
MINUTOS            805
IGNORADO           269
SIN REGISTRO       138
SEGUNDOS            33
Name: TIEMPO EDAD, dtype: int64

In [44]:
fal_sin["TIEMPO EDAD"].value_counts()

AÑOS            527537
DIAS              7541
MESES             7047
HORAS             2641
MINUTOS            805
IGNORADO           269
SIN REGISTRO       138
SEGUNDOS            33
Name: TIEMPO EDAD, dtype: int64

In [45]:
def TIEMPO_EDAD_FE_ANIO(x) :
    if (x=="AÑOS"):
      return 1
    else:
      return 0        

In [46]:
fal_sin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546011 entries, 0 to 546010
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   TIPO SEGURO             546011 non-null  object
 1   SEXO                    546011 non-null  object
 2   EDAD                    546011 non-null  object
 3   TIEMPO EDAD             546011 non-null  object
 4   ESTADO CIVIL            546011 non-null  object
 5   NIVEL DE INSTRUCCIÓN    546011 non-null  object
 6   COD# UBIGEO DOMICILIO   546011 non-null  object
 7   PAIS DOMICILIO          546011 non-null  object
 8   DEPARTAMENTO DOMICILIO  546011 non-null  object
 9   PROVINCIA DOMICILIO     546011 non-null  object
 10  DISTRITO DOMICILIO      546011 non-null  object
 11  FECHA                   546011 non-null  object
 12  AÑO                     546011 non-null  int64 
 13  MES                     546011 non-null  int64 
 14  TIPO LUGAR              546011 non-n

In [47]:
fal_sin["factor"]=fal_sin["TIEMPO EDAD"].apply(TIEMPO_EDAD_FE_ANIO)

In [48]:
fal_sin.loc[fal_sin["EDAD"]=="SIN REGISTRO","EDAD"]=str(fal_sin.loc[fal_sin["EDAD"]!="SIN REGISTRO","EDAD"].astype(int).mean())

In [49]:
fal_sin["EDAD"]=fal_sin["EDAD"].astype(float).astype(int)

In [50]:
fal_sin["EDAD"].describe()

count    546011.000000
mean         66.322334
std          23.435579
min           0.000000
25%          55.000000
50%          72.000000
75%          83.000000
max        3017.000000
Name: EDAD, dtype: float64

In [51]:
fal_sin["TIEMPO"]=fal_sin["EDAD"]*fal_sin["factor"]

In [52]:
fal_sin["TIEMPO"]

0          4
1         86
2         38
3         64
4         20
          ..
546006    82
546007    72
546008    86
546009    89
546010     0
Name: TIEMPO, Length: 546011, dtype: int64

In [53]:
fal_sin.loc[(fal_sin["DEBIDO A (CAUSA A)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA B)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA C)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA D)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA E)"].str.contains("covid", case=False)) |
        (fal_sin["DEBIDO A (CAUSA F)"].str.contains("covid", case=False)) ,"Target"]=1
                

In [54]:
fal_sin["Target"]=fal_sin["Target"].fillna(0)

In [55]:
fal_sin["Target"]

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
546006    1.0
546007    1.0
546008    0.0
546009    0.0
546010    0.0
Name: Target, Length: 546011, dtype: float64

In [56]:
fal_sin.columns

Index(['TIPO SEGURO', 'SEXO', 'EDAD', 'TIEMPO EDAD', 'ESTADO CIVIL',
       'NIVEL DE INSTRUCCIÓN', 'COD# UBIGEO DOMICILIO', 'PAIS DOMICILIO',
       'DEPARTAMENTO DOMICILIO', 'PROVINCIA DOMICILIO', 'DISTRITO DOMICILIO',
       'FECHA', 'AÑO', 'MES', 'TIPO LUGAR', 'INSTITUCION', 'MUERTE VIOLENTA',
       'NECROPSIA', 'DEBIDO A (CAUSA A)', 'CAUSA A (CIE-X)',
       'DEBIDO A (CAUSA B)', 'CAUSA B (CIE-X)', 'DEBIDO A (CAUSA C)',
       'CAUSA C (CIE-X)', 'DEBIDO A (CAUSA D)', 'CAUSA D (CIE-X)',
       'DEBIDO A (CAUSA E)', 'CAUSA E (CIE-X)', 'DEBIDO A (CAUSA F)',
       'CAUSA F (CIE-X)', 'TIPO SEGURO_FE', 'factor', 'TIEMPO', 'Target'],
      dtype='object')

In [57]:
fal_sin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546011 entries, 0 to 546010
Data columns (total 34 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   TIPO SEGURO             546011 non-null  object 
 1   SEXO                    546011 non-null  object 
 2   EDAD                    546011 non-null  int64  
 3   TIEMPO EDAD             546011 non-null  object 
 4   ESTADO CIVIL            546011 non-null  object 
 5   NIVEL DE INSTRUCCIÓN    546011 non-null  object 
 6   COD# UBIGEO DOMICILIO   546011 non-null  object 
 7   PAIS DOMICILIO          546011 non-null  object 
 8   DEPARTAMENTO DOMICILIO  546011 non-null  object 
 9   PROVINCIA DOMICILIO     546011 non-null  object 
 10  DISTRITO DOMICILIO      546011 non-null  object 
 11  FECHA                   546011 non-null  object 
 12  AÑO                     546011 non-null  int64  
 13  MES                     546011 non-null  int64  
 14  TIPO LUGAR          

In [58]:
data_weka=fal_sin[['TIPO SEGURO', 
'SEXO', 
'ESTADO CIVIL',
'NIVEL DE INSTRUCCIÓN', 
'TIPO SEGURO_FE', 
'TIEMPO',
'PAIS DOMICILIO',
'DEPARTAMENTO DOMICILIO', 
'PROVINCIA DOMICILIO', 
'DISTRITO DOMICILIO',
'AÑO', 
'MES', 
'TIPO LUGAR', 
'INSTITUCION', 
'MUERTE VIOLENTA',
'NECROPSIA', 
'Target']]

In [59]:
data_weka.isnull().sum()

TIPO SEGURO               0
SEXO                      0
ESTADO CIVIL              0
NIVEL DE INSTRUCCIÓN      0
TIPO SEGURO_FE            0
TIEMPO                    0
PAIS DOMICILIO            0
DEPARTAMENTO DOMICILIO    0
PROVINCIA DOMICILIO       0
DISTRITO DOMICILIO        0
AÑO                       0
MES                       0
TIPO LUGAR                0
INSTITUCION               0
MUERTE VIOLENTA           0
NECROPSIA                 0
Target                    0
dtype: int64

In [60]:
data_weka.columns

Index(['TIPO SEGURO', 'SEXO', 'ESTADO CIVIL', 'NIVEL DE INSTRUCCIÓN',
       'TIPO SEGURO_FE', 'TIEMPO', 'PAIS DOMICILIO', 'DEPARTAMENTO DOMICILIO',
       'PROVINCIA DOMICILIO', 'DISTRITO DOMICILIO', 'AÑO', 'MES', 'TIPO LUGAR',
       'INSTITUCION', 'MUERTE VIOLENTA', 'NECROPSIA', 'Target'],
      dtype='object')

In [61]:
data_weka["Target"].value_counts(dropna=False)

0.0    478374
1.0     67637
Name: Target, dtype: int64

In [62]:
data_weka["Target"]=data_weka["Target"].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in ['TIPO SEGURO', 'SEXO', 'ESTADO CIVIL','NIVEL DE INSTRUCCIÓN',  'PAIS DOMICILIO','DEPARTAMENTO DOMICILIO', 'PROVINCIA DOMICILIO', 'DISTRITO DOMICILIO','TIPO LUGAR', 'INSTITUCION', 'MUERTE VIOLENTA','NECROPSIA', 'TIPO SEGURO_FE']:
  data_weka[i] = labelencoder.fit_transform(data_weka[i])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [64]:
train_full=data_weka.copy()

In [65]:
features=[ x for x in train_full.columns if x not in ['Target']]

In [66]:
target="Target"

In [67]:
train_full.columns

Index(['TIPO SEGURO', 'SEXO', 'ESTADO CIVIL', 'NIVEL DE INSTRUCCIÓN',
       'TIPO SEGURO_FE', 'TIEMPO', 'PAIS DOMICILIO', 'DEPARTAMENTO DOMICILIO',
       'PROVINCIA DOMICILIO', 'DISTRITO DOMICILIO', 'AÑO', 'MES', 'TIPO LUGAR',
       'INSTITUCION', 'MUERTE VIOLENTA', 'NECROPSIA', 'Target'],
      dtype='object')

In [69]:
gc.collect()

188

In [70]:
kf_previo=StratifiedKFold(n_splits=5,random_state=256,shuffle=True)

i=1

r=[]

test_probs = []

importancias=pd.DataFrame()

importancias['variable']=features

for train_index,test_index in kf_previo.split(train_full,train_full[target]):

    lgb_train = lgb.Dataset(train_full.loc[train_index,features].values,train_full.loc[train_index,target].values.ravel())
    lgb_eval = lgb.Dataset(train_full.loc[test_index,features].values,train_full.loc[test_index,target].values.ravel(), reference=lgb_train)

    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': { 'auc'},
        "max_depth":4,
        "num_leaves":10,
        'learning_rate': 0.1,
    "min_child_samples": 100,
        'feature_fraction': 0.5,
     "bagging_freq":1,
        'bagging_fraction': 0.9,
        "lambda_l1":1,
        "lambda_l2":1,
        #"scale_pos_weight":24,

        'verbose': 1    
    }




    lgbm3 = lgb.train(params,lgb_train,num_boost_round=13100,valid_sets=lgb_eval,early_stopping_rounds=50,verbose_eval=25)#,categorical_feature=cat_ind)
    
    #test["TARGET_FOLD"+str(i)]=
    
    #test_probs.append(pd.Series(lgbm3.predict(test[features].values, num_iteration=lgbm3.best_iteration)))
    
    
    importancias['gain_'+str(i)]=lgbm3.feature_importance(importance_type="gain")

    
    print ("Fold_"+str(i))
    a= (roc_auc_score(train_full.loc[test_index,target],lgbm3.predict(train_full.loc[test_index,features].values, num_iteration=lgbm3.best_iteration)))
    r.append(a)
    print (a)
    print ("")
    
    i=i+1

print ("mean: "+str(np.mean(np.array(r))))
print ("std: "+str(np.std(np.array(r))))
#test_probs = pd.concat(test_probs, axis=1).mean(axis=1)

Training until validation scores don't improve for 50 rounds.
[25]	valid_0's auc: 0.953386
[50]	valid_0's auc: 0.9565
[75]	valid_0's auc: 0.957734
[100]	valid_0's auc: 0.958499
[125]	valid_0's auc: 0.959017
[150]	valid_0's auc: 0.959456
[175]	valid_0's auc: 0.959816
[200]	valid_0's auc: 0.960153
[225]	valid_0's auc: 0.960564
[250]	valid_0's auc: 0.960721
[275]	valid_0's auc: 0.961006
[300]	valid_0's auc: 0.961191
[325]	valid_0's auc: 0.961403
[350]	valid_0's auc: 0.961598
[375]	valid_0's auc: 0.96174
[400]	valid_0's auc: 0.961802
[425]	valid_0's auc: 0.961958
[450]	valid_0's auc: 0.962085
[475]	valid_0's auc: 0.962208
[500]	valid_0's auc: 0.9623
[525]	valid_0's auc: 0.962411
[550]	valid_0's auc: 0.962536
[575]	valid_0's auc: 0.962628
[600]	valid_0's auc: 0.962686
[625]	valid_0's auc: 0.962769
[650]	valid_0's auc: 0.962822
[675]	valid_0's auc: 0.962894
[700]	valid_0's auc: 0.962948
[725]	valid_0's auc: 0.962997
[750]	valid_0's auc: 0.963052
[775]	valid_0's auc: 0.963093
[800]	valid_0's 

In [72]:

w=[x for x in importancias.columns if 'gain_' in x]

importancias['gain-avg']=importancias[w].mean(axis=1)

importancias=importancias.sort_values('gain-avg',ascending=False).reset_index(drop=True)

In [73]:
importancias

,variable,gain_1,gain_2,gain_3,gain_4,gain_5,gain-avg
0,AÑO,485870.741778,484884.107778,486792.896745,486320.515530,487855.059704,486344.664307
1,MES,138319.766388,138441.417628,137380.591886,138590.002064,138098.482009,138166.051995
2,INSTITUCION,101259.758149,103072.145314,102684.936413,103344.371681,104477.538002,102967.749912
3,NIVEL DE INSTRUCCIÓN,43353.296092,42455.676000,44346.336248,43375.953208,42565.098694,43219.272049
4,TIPO LUGAR,38835.113842,39370.297935,40104.633745,39153.058889,39753.629682,39443.346818
5,TIEMPO,32625.203456,33191.512068,33591.301462,33027.910633,34590.001210,33405.185766
6,NECROPSIA,31625.710175,32022.876367,31295.650838,31728.859321,31642.216499,31663.062640
7,DEPARTAMENTO DOMICILIO,21403.562796,21889.510326,21983.249895,23039.293078,23739.975506,22411.118320
8,PROVINCIA DOMICILIO,18587.622044,18040.012477,20005.511141,19675.127608,18607.199447,18983.094543
9,DISTRITO DOMICILIO,8955.239150,9537.085954,10935.093841,10783.521191,11158.283071,10273.844641


In [ ]:
"AÑO","MES","INSTITUCION","NIVEL DE INSTRUCCIÓN","TIPO LUGAR","TIEMPO","NECROPSIA","DEPARTAMENTO DOMICILIO","PROVINCIA DOMICILIO","DISTRITO DOMICILIO","SEXO"	